In [58]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import getpass
import datetime

In [59]:
table = pd.read_csv("final_food_iati_capital.csv")
table

,label,price,city,Countries,code
0,Meal for 2 People Mid-range Restaurant Three-c...,10.30,Kabul,Afghanistan,KBL
1,Meal for 2 People Mid-range Restaurant Three-c...,34.27,Tirana,Albania,TIA
2,Meal for 2 People Mid-range Restaurant Three-c...,21.75,Algiers,Algeria,ALG
3,Meal for 2 People Mid-range Restaurant Three-c...,50.00,Andorra-La-Vella,Andorra,ALG
4,Meal for 2 People Mid-range Restaurant Three-c...,65.30,Luanda,Angola,LAD
...,...,...,...,...,...
144,Meal for 2 People Mid-range Restaurant Three-c...,30.49,Tashkent,Uzbekistan,TAS
145,Meal for 2 People Mid-range Restaurant Three-c...,50.81,Caracas,Venezuela,CCS
146,Meal for 2 People Mid-range Restaurant Three-c...,18.66,Hanoi,Vietnam,HAN
147,Meal for 2 People Mid-range Restaurant Three-c...,16.09,Lusaka,Zambia,LUN


In [60]:
date_b = input('Hi, are you free to travel? When are you planning to fly? (YYYY-MM-DD)?')
date_r = input('What date you plane to get return?')
budget = float(input('What is you budget?'))
budget_food = budget*0.33

travel_time = (datetime.datetime.strptime(date_r,'%Y-%m-%d')-datetime.datetime.strptime(date_b,'%Y-%m-%d')).days

In [61]:
""" CHECK FOODING"""
table_budget = table[table['price']*2*travel_time < budget_food]
table_budget

,label,price,city,Countries,code
0,Meal for 2 People Mid-range Restaurant Three-c...,10.30,Kabul,Afghanistan,KBL
1,Meal for 2 People Mid-range Restaurant Three-c...,34.27,Tirana,Albania,TIA
2,Meal for 2 People Mid-range Restaurant Three-c...,21.75,Algiers,Algeria,ALG
5,Meal for 2 People Mid-range Restaurant Three-c...,25.41,Buenos-Aires,Argentina,LAD
9,Meal for 2 People Mid-range Restaurant Three-c...,29.89,Baku,Azerbaijan,BAK
...,...,...,...,...,...
137,Meal for 2 People Mid-range Restaurant Three-c...,16.41,Ankara,Turkey,ANK
139,Meal for 2 People Mid-range Restaurant Three-c...,26.68,Kampala,Uganda,ASB
144,Meal for 2 People Mid-range Restaurant Three-c...,30.49,Tashkent,Uzbekistan,TAS
146,Meal for 2 People Mid-range Restaurant Three-c...,18.66,Hanoi,Vietnam,HAN


In [70]:
table_budget.sample(frac=1)
code_list = list(table_budget['code'].head(5))

In [71]:
""" CHECK FLIGHTS"""

from requests.auth import HTTPBasicAuth
import getpass
url="https://developers.amadeus.com/my-apps/orbitfly"
password = 'Shameonyou1@'
response = requests.post(url=url,auth=HTTPBasicAuth("Bewek Bastola",password))
response=requests.post(url=url)

In [72]:
from amadeus import ResponseError, Location, Client
amadeus = Client(
    client_id='IBcYKGqcwh0hzDXldNCWk9l4iZP9AkhH',
    client_secret='TYxCgbyLitTM6ydO'
)

dict= {}
final_list=[]
for airp in code_list:
    response = amadeus.shopping.flight_offers_search.get(
            originLocationCode='LIS',
            destinationLocationCode= airp,
            departureDate='2022-11-01',
            returnDate='2022-12-01',
            adults=2)
    data = response.data

    prices = [i['price'] for i in data]
    prices_df = pd.DataFrame.from_dict(prices)

    dict = {'airp_code' : airp, 'min_price' : prices_df['total'].min()}
    final_list.append(dict)

In [73]:
flights_df = pd.DataFrame(final_list)
flights_df

,airp_code,min_price
0,KBL,2865.44
1,TIA,2121.76
2,ALG,1192.32
3,LAD,10355.24
4,BAK,1493.22


In [74]:
food_flight_df = pd.merge(left=flights_df, right=table, how='inner', left_on='airp_code', right_on='code').rename(columns={'price':'food_price','min_price':'flight_price','Countries':'countries'}).drop(columns=['label', 'city', 'code'])
food_flight_df

,airp_code,flight_price,food_price,countries
0,KBL,2865.44,10.30,Afghanistan
1,TIA,2121.76,34.27,Albania
2,ALG,1192.32,21.75,Algeria
3,ALG,1192.32,50.00,Andorra
4,LAD,10355.24,65.30,Angola
5,LAD,10355.24,25.41,Argentina
6,BAK,1493.22,29.89,Azerbaijan


In [75]:
accom_df = pd.read_csv("acommodation.csv")
accom_df

,country,capital,price
0,Afghanistan,Kabul,229.50
1,Albania,Tirana,82.28
2,Algeria,Algiers,77.87
3,Andorra,Andorra la Vella,77.18
4,Angola,Luanda,101.99
...,...,...,...
192,Venezuela,Caracas,136.34
193,Vietnam,Hanoi,136.04
194,Yemen,Sana'a,136.04
195,Zambia,Lusaka,135.89


In [76]:
final_df = pd.merge(left=food_flight_df, right=accom_df, how='inner', left_on='countries', right_on='country').rename(columns={'price':'accom_price'})
final_df

,airp_code,flight_price,food_price,countries,country,capital,accom_price
0,KBL,2865.44,10.30,Afghanistan,Afghanistan,Kabul,229.50
1,TIA,2121.76,34.27,Albania,Albania,Tirana,82.28
2,ALG,1192.32,21.75,Algeria,Algeria,Algiers,77.87
3,ALG,1192.32,50.00,Andorra,Andorra,Andorra la Vella,77.18
4,LAD,10355.24,65.30,Angola,Angola,Luanda,101.99
5,LAD,10355.24,25.41,Argentina,Argentina,Buenos Aires,137.52
6,BAK,1493.22,29.89,Azerbaijan,Azerbaijan,Baku,136.53


In [77]:
final_df ["column"] = final_df["food_price"].astype(float) + final_df["flight_price"].astype(float) + final_df["accom_price"].astype(float)

In [78]:
def ok_nok(x):
    if x <= budget:
        return 'OK'
    else:
        return 'NOK'

final_df['decision'] = final_df['column'].apply(ok_nok)
final_df

,airp_code,flight_price,food_price,countries,country,capital,accom_price,column,decision
0,KBL,2865.44,10.30,Afghanistan,Afghanistan,Kabul,229.50,3105.24,NOK
1,TIA,2121.76,34.27,Albania,Albania,Tirana,82.28,2238.31,OK
2,ALG,1192.32,21.75,Algeria,Algeria,Algiers,77.87,1291.94,OK
3,ALG,1192.32,50.00,Andorra,Andorra,Andorra la Vella,77.18,1319.50,OK
4,LAD,10355.24,65.30,Angola,Angola,Luanda,101.99,10522.53,NOK
5,LAD,10355.24,25.41,Argentina,Argentina,Buenos Aires,137.52,10518.17,NOK
6,BAK,1493.22,29.89,Azerbaijan,Azerbaijan,Baku,136.53,1659.64,OK


In [79]:
final_df["decision"].value_counts()

OK     4
NOK    3
Name: decision, dtype: int64

In [80]:
c= final_df[final_df["decision"] == 'OK']

In [94]:
c.rename(columns={'column':'final cost'}, inplace=True)

if len(c) == 0:
    print("Upsss, you don't have money enough... looks like you'll have to wait for you data job!")
else:
    print("""This are your options, say thanks! \n
            Day of departure: """, date_b, """
            Day of return: """, date_r, """
            Number of happy days: """, travel_time, """
            Original budget: """, budget,"""€ \n""")
    display(c[["country","capital","final cost"]])


This are your options, say thanks! 

            Day of departure:  2022-12-23 
            Day of return:  2023-01-03 
            Number of happy days:  11 
            Original budget:  2500.0 € 



C:\Users\inesa\AppData\Local\Temp\ipykernel_7696\3101387308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c.rename(columns={'column':'final cost'}, inplace=True)


,country,capital,final cost
1,Albania,Tirana,2238.31
2,Algeria,Algiers,1291.94
3,Andorra,Andorra la Vella,1319.50
6,Azerbaijan,Baku,1659.64
